Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 

# MONAI Label - Lung Nodule Detection - 3D Slicer Integration

***The Detection Task Workflow in MONAI Label***

In this notebook, we provide a monaibundle app usecase example with lung nodule detection. The first detection task that can be used within MONAI Label.

3D Slicer as the client viewer, we show how MONAI Label workflow serves as an interactive AI-Assisted tool for box-based ROI detection labeling.

**Lung Nodule Detection Bundle:**

![workflow](./figures/monailabel_detection_3dslicer/3dslicer_demo.png)


**Method:** Lung Nodule Detection Networks

The workflow of lung nodule detection with CT scans is from RetinaNet (Lin, Tsung-Yi, et al. "Focal loss for dense object detection." ICCV 2017. https://arxiv.org/abs/1708.02002).

![workflow](./figures/monailabel_detection_3dslicer/detection_networks.png)

## Setup environment

MONAI Label is pre-built within the MONAI Toolkit. Download monaibundle app

### Prerequisites
- **Install MONAI Label**

In [ ]:
!pip install monailabel

- **Get latest MONAI Label Slicer plugin** (Optional)

If you are using older release of 3D Slicer MONAI Label plugin, you can update to the latest plugin which supports ***drawing ROI nodes from MONAI Label server prediction***.

Or to use the developer mode of MONAI Label plugin by:

 - Pull latest MONAI Label repo:
 ```bash
 git clone https://github.com/Project-MONAI/MONAILabel.git
 ```
 - Go to `MONAILabel/plugins/slicer/`
 - Follow the Readme page to use developer mode of Slicer plugin.
 https://github.com/Project-MONAI/MONAILabel/tree/main/plugins/slicer#install-plugin-in-developer-mode
    - Install Plugin in Developer Mode
        - Open 3D Slicer: Go to Edit -> Application Settings -> Modules -> Additional Module Paths
        - Add New Module Path: <FULL_PATH>/plugins/slicer/MONAILabel
        - Restart 3D Slicer
        

### 1.1 Download monaibundle app
Sample-app with built-in versions are in the Toolkit, pull the monaibundle app using monailabel API 

In [2]:
!monailabel apps --download --name monaibundle --output apps

Using PYTHONPATH=/home/yucheng:

monaibundle is copied at: /home/yucheng/yucheng/2023/detection_totalsegment_tutorial/tutorials/monailabel/apps/monaibundle


### 1.2 Download sample data
The lung nodule detection model is trained and evaluated using [LUNA16](https://luna16.grand-challenge.org/Home/) dataset.

LUNA16 is a public dataset of CT lung nodule detection. Using raw CT scans, the goal is to identify locations of possible nodules and to assign a probability for being a nodule to each location.

Disclaimer: We are not the host of the data. Please make sure to read the requirements and usage policies of the data and give credit to the authors of the dataset! We acknowledge the National Cancer Institute and the Foundation for the National Institutes of Health, and their critical role in the creation of the free publicly available LIDC/IDRI Database used in this study.

 - If you are going to use full dataset of LUNA16, please refer to the dataset link, download the data, create and preprocess the images following [this page](        https://github.com/Project-MONAI/model-zoo/tree/dev/models/lung_nodule_ct_detection#data 
).
 
 - In this tutorial, we prepared a sample subset, resampled and ready to use. The subset is only for demonstration. Download [here](https://github.com/Project-MONAI/MONAILabel/releases/download/data/detection_dataset.zip).
 
   Or use these commands to download and extract directly.

```bash
mkdir datasets
wget "https://github.com/Project-MONAI/MONAILabel/releases/download/data/detection_dataset.zip" -O datasets/detection_dataset.zip
unzip datasets/detection_dataset.zip -d datasets/detection_dataset
```
 
 - The detection model also supports MSD Task06_Lung dataset for inference. Note this dataset is created for nodule segmentation task. We do not use the segmentation mask, only the images for inference. Use following monailabel API to automatically download the dataset.

In [ ]:
!monailabel datasets --download --name Task06_Lung --output datasets

### 1.3 Starting MONAI Label Server

Specify the bundle name in --conf models <BUNDLENAME> argument. For example: lung_nodule_ct_detection


In [ ]:
!monailabel start_server --app apps/monaibundle --studies datasets/Task06_Lung/imagesTr --conf models lung_nodule_ct_detection

### 1.4 (Optional) Modify Detection Parameters in the Bundle

The lung_nodule_ct_detection bundle can set box prediction probability threshold. In `apps/monaibundle/model/lung_nodule_ct_detection_v0.5.0/configs/inference.json`, it's define here `score_thresh=0.02`: 

```json
    "detector_ops": [
        "$@detector.set_target_keys(box_key='box', label_key='label')",
        "$@detector.set_box_selector_parameters(score_thresh=0.02,topk_candidates_per_level=1000,nms_thresh=0.22,detections_per_img=300)",
        "$@detector.set_sliding_window_inferer(roi_size=@infer_patch_size,overlap=0.25,sw_batch_size=1,mode='constant',device='cpu')"
    ],

```

This parameter will impact the number of boxes in the final prediction output. Then show all boxes ROIs in 3D Slicer.

If users want to limit the number of predicted boxes showing on 3D Slicer, users can set higher `score_thresh`, e.g., `score_thresh=0.5`, `score_thresh=0.6`, or even higher. 

## 2. Auto Box Detection with 3D Slicer

Below is a step-by-step tutorial on using 3D Slicer for auto lung nodule detection. Pre-trained model is provided by the bundle and loaded by MONAI Label server.


**Start 3D Slicer and MONAI Label Plugin**: the tutorial starts with monaibundle app in the 3D Slicer, please refer to the ["Hello World" tutorial](monailabel_radiology_3dslicer.ipynb) for detailed instructions on installing 3D Slicer and MONAI Label plugin.

**Important**: If your MONAI Label plugin is from older release, please refer to [Prerequisites](#Prerequisites) in this notebook to update and use latest MONAI Label plugin. The detection task needs to draw ROI nodes to 3D Slicer markups.

### 2.1 Select Model and Fetch Data

- Select the bundle model loaded with MONAI Label server, **lung_nodule_ct_detection** in this tutorial. Start MONAI Label server will automatically download the bundle.


click **next sample** to load image data from datastore.

![slicer1](./figures/monailabel_detection_3dslicer/detection1.png)


### 2.2 Run Auto Inference

- MONAI Label has loaded the pre-trained weights within the bundle for multi-organ segmentation task, click **run** to do inference on the currently loaded data. 

- Note: the pre-trained models are in the bundle's "**models**" folder. For instance, in this usecase, a downloaded pre-trained model is saved at "**apps/monaibundle/model/lung_nodule_ct_detection/models/model.pt**".

- Note: the detection model uses RetinaNet, please reserve at least ~12G memory for inference and training, modify the bundle config file if out of memory.
- Users can move the axis and slices to view the correct location of segmentation.

Users can monitor the logs in the MONAI Label server terminal

![slicer1](./figures/monailabel_detection_3dslicer/detection2.png)


### 2.3 Edit Annotations and Submit Label

- Users can edit the inference labels manually using 3D Slicer ROI box Editor, click the **Markups** tab on the top, the tools for adjusting ROIs. Once done annotation, users can go back to MONAI Label plugin and click **Submit Label** to save the ground truth label to the file system.

- Note: the final annotations will be saved to "**labels/final**" folder in the study dataset, for example, in this usecase, the ground truth label will be saved to "**datasets/Task06_Lung/imagesTr/labels/final**"

- The saved annotation will be in JSON file with the 3D Slicer ROINode format. It can be loaded with 3D Slicer.

![slicer1](./figures/monailabel_detection_3dslicer/detection3.png)


## 3. Active Learning with 3D Slicer

Active learning and interactive fine-tuning models have highlighted features in MONAI Label, users can train their models anytime when new annotated labels are saved. Click **Train** button, MONAI Label server will fetch the saved final ground truth label and fine-tune the prior model. 

- Sample of annotated box coordinate in JSON file. 

![slicer1](./figures/monailabel_detection_3dslicer/detection4.png)


Users can monitor the logs in the MONAI Label server terminal

- The ROINode box annotation will be converted to the model-compatible ground truth label automatically.

![slicer1](./figures/monailabel_detection_3dslicer/detection5.png)

Same to the Radiology app, users can do auto segmentation using the latest fine-tuned model. 
The active learning process will select unlabeled batch data.

The next image is selected, already trained images will be marked as labeled data, then won't be selected for the next labeling batch. Active learning strategies such as "first/random" will be used for selecting which unlabeled data to fetch. 



## 4. Repeat Until All Data are Annotated and Trained 

Repeated fetch data and active learning process to section 2.2, until all unlabeled data are annotated and trained. 

For each training loop, the new best metric model will be saved in "**model/model.pt**", in this usecase, "**apps/monaibundle/lung_nodule_ct_detection/model/model.pt**" is saved.

## 5. Conclusion

This tutorial demonstrates MONAI Label usage with monaibundle app, the lung nodule ct detection task is introduced. Active learning process, auto inference, detection editor, submit labels, and save models are included with the bundle. The monaibundle app highlights the robust model deployment for MONAI Label.